In [14]:
#-------------------------------------------------------------------------------
# elftools example: dwarf_location_info.py
#
# Examine DIE entries which have either location list values or location
# expression values and decode that information.
#
# Location information can either be completely contained within a DIE
# (using 'DW_FORM_exprloc' in DWARFv4 or 'DW_FORM_block1' in earlier
# versions) or be a reference to a location list contained within
# the .debug_loc section (using 'DW_FORM_sec_offset' in DWARFv4 or
# 'DW_FORM_data4' / 'DW_FORM_data8' in earlier versions).
#
# The LocationParser object parses the DIE attributes and handles both
# formats.
#
# The directory 'test/testfiles_for_location_info' contains test files with
# location information represented in both DWARFv4 and DWARFv2 forms.
#
# Eli Bendersky (eliben@gmail.com)
# This code is in the public domain
#-------------------------------------------------------------------------------
from __future__ import print_function
import sys

# If pyelftools is not installed, the example can also run from the root or
# examples/ dir of the source distribution.
sys.path[0:0] = ['.', '..']

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)



def show_loclist(loclist, dwarfinfo, indent, cu_offset):
    """ Display a location list nicely, decoding the DWARF expressions
        contained within.
    """
    d = []
    for loc_entity in loclist:
        if isinstance(loc_entity, LocationEntry):
            d.append('%s <<%s>>' % (
                loc_entity,
                describe_DWARF_expr(loc_entity.loc_expr, dwarfinfo.structs, cu_offset)))
        else:
            d.append(str(loc_entity))
    return '\n'.join(indent + s for s in d)

def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        # get_dwarf_info returns a DWARFInfo context object, which is the
        # starting point for all DWARF-based processing in pyelftools.
        dwarfinfo = elffile.get_dwarf_info()

        # The location lists are extracted by DWARFInfo from the .debug_loc
        # section, and returned here as a LocationLists object.
        location_lists = dwarfinfo.location_lists()

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        # Create a LocationParser object that parses the DIE attributes and
        # creates objects representing the actual location information.
        loc_parser = LocationParser(location_lists)

        for CU in dwarfinfo.iter_CUs():
            # DWARFInfo allows to iterate over the compile units contained in
            # the .debug_info section. CU is a CompileUnit object, with some
            # computed attributes (such as its offset in the section) and
            # a header which conforms to the DWARF standard. The access to
            # header elements is, as usual, via item-lookup.
            print('  Found a compile unit at offset %s, length %s' % (
                CU.cu_offset, CU['unit_length']))

            # A CU provides a simple API to iterate over all the DIEs in it.
            die_depth = 0
            for DIE in CU.iter_DIEs():
                # Go over all attributes of the DIE. Each attribute is an
                # AttributeValue object (from elftools.dwarf.die), which we
                # can examine.
                # print('\n',">"*30,DIE,'\n'*2 , '_'*5+'\n')
                
                

                    
                    
                print("\n\nTOTAL", len(DIE.attributes.values()))
                
                
                print("DIE DEPTH",die_depth )
                print("child" , DIE.has_children )
                print(" die.tag_ >", DIE.tag)
                print("die.abbrev_code", DIE.abbrev_code)
                print("die.offset",DIE.offset)
                
                
                for attr in DIE.attributes:
                    print(attr)
                for attr in DIE.attributes.values():
                    print('\t\t',attr )
                    
                print( "_"*40,'\n'*4)
                
                if DIE.is_null(): #https://chromium.googlesource.com/chromiumos/third_party/pyelftools/+/25a77f7738d7fe824f2ed4d33a123136b9d8e88a/scripts/readelf.py
                    die_depth -= 1
                    print("+"*50)
                    continue
                if DIE.has_children:
                    die_depth += 1
                    
            # break



filename = './../binaries/c_many/stacktest'
process_file(filename)

Processing file: ./../binaries/c_many/stacktest
  Found a compile unit at offset 0, length 416


TOTAL 8
DIE DEPTH 0
child True
 die.tag_ > DW_TAG_compile_unit
die.abbrev_code 8
die.offset 12
DW_AT_producer
DW_AT_language
DW_AT_name
DW_AT_comp_dir
DW_AT_GNU_pubnames
DW_AT_low_pc
DW_AT_high_pc
DW_AT_stmt_list
		 AttributeValue(name='DW_AT_producer', form='DW_FORM_strp', value=b'GNU C17 11.3.0 -mtune=generic -march=x86-64 -gdwarf -gpubnames -fasynchronous-unwind-tables -fstack-protector-strong -fstack-clash-protection -fcf-protection', raw_value=86, offset=13)
		 AttributeValue(name='DW_AT_language', form='DW_FORM_data1', value=29, raw_value=29, offset=17)
		 AttributeValue(name='DW_AT_name', form='DW_FORM_line_strp', value=b'stack.c', raw_value=36, offset=18)
		 AttributeValue(name='DW_AT_comp_dir', form='DW_FORM_line_strp', value=b'/home/nahid/reverse/binaries/c_many', raw_value=0, offset=22)
		 AttributeValue(name='DW_AT_GNU_pubnames', form='DW_FORM_flag_present', value=True, raw_valu